In [1]:
import tomotopy as tp
from gensim import corpora
from gensim.models import CoherenceModel
from tqdm import tqdm
#plot
import matplotlib
matplotlib.use("TkAgg")  # Define o backend ANTES de importar pyplot

import matplotlib.pyplot as plt  # Só importar depois de definir o backend

from nltk import tokenize
import pandas as pd

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
from unidecode import unidecode


In [2]:
df = pd.read_csv('../datasets/fakeWhatsApp.BR_2022.csv')

In [3]:
len(df)

598971

In [4]:
df.head()

,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,trava_zap,text_content_anonymous,dataset_info_id,date_system,score_sentiment,score_misinformation,id_message,id_persona,message_type,messenger,media_name,media_md5
0,2022-10-10 18:20:24,9d737b3c9387855139bbad2311cc5709,0638569ee76dac58f59dcac20463c955,NaN,NaN,NaN,False,False,False,☝️\nHoje que eu fique sabendo do CANAL LULA FL...,5,2022-10-10 18:20:25.000937,-0.7003,0.843775,F7023FFB06C429A2C166922849A35ED8,558594228826.0:12@s.whatsapp.net,Texto,whatsapp,NaN,NaN
1,2022-10-10 22:02:58,1660a60f661754d2802ca53296e25be8,a5910d5cc1c830ade9eb4dd00f15ff6a,NaN,NaN,NaN,False,False,False,Mais pra que isso não aconteça nois temos quê ...,5,2022-10-10 22:02:58.4682,-0.2960,NaN,A9FAC78070C144890D181EF415B90CAD,558594228826.0:12@s.whatsapp.net,TextoExtendido,whatsapp,NaN,NaN
2,2022-10-11 00:39:31,c882172d447798d74915973ac83eba68,b84dfe2d1599b82768dcdecce7e6bb23,d2e0ec59ffd9f84764f5b147725d7196.oga,audio/ogg; codecs=opus,NaN,True,False,False,NaN,5,2022-10-11 00:39:33.445125,NaN,NaN,737948BE86D450A426470794F91BC80D,558594228826.0:12@s.whatsapp.net,Audio,whatsapp,NaN,d2e0ec59ffd9f84764f5b147725d7196
3,2022-10-10 23:36:19,91e3c22c08b24ba01ac4524d77bcb1da,addb88a34374d43aa9ecd4df7359ce39,NaN,NaN,NaN,False,False,False,‎Acesse este link para entrar no meu grupo do ...,5,2022-10-10 23:36:19.724987,-0.1531,NaN,439A91ADD8F355CD23C4BB107A5E88BB,558594228826.0:12@s.whatsapp.net,TextoExtendido,whatsapp,NaN,NaN
4,2022-10-10 23:40:12,77c1a8a31fee269db258a028a61f0b88,a5831b109d4d95fae8ee0ce464e48a6a,bb80cd530332bb6b95c34e719992d249.f4v,video/mp4,NaN,True,False,False,NaN,5,2022-10-10 23:40:14.397495,NaN,NaN,092203A082AC3DFB2A4933F60453AEB8,558594228826.0:12@s.whatsapp.net,Video,whatsapp,NaN,bb80cd530332bb6b95c34e719992d249


In [5]:
ids_para_remover = ['6ef561ec0f448afcd7b3751124bb0712','d3e678a0ba0e1485548260a7c4599152',
                    'ac5703154484de05336af617455ca55e','a39edbd64d378226ffa60433649a0acf',
                    'c66d0d4ae5a4b281bff67e1fa4fbd6ba','819bbc872ed6d81f44d746b710eecf06',
                    'ec94da4d54f9a5693e88fa582926be53','6a38c72316d87c028dfd66c10442476b',
                    '94099e1e46f129856541e2b3640896d1','8f367d1693fff47218603fa47ded525c',
                    'c09caffee0d1bd30926dea9df25dc88f']

In [6]:
# Filtrar o DataFrame para remover as linhas com esses IDs
df_filtrado = df[~df['id_member_anonymous'].isin(ids_para_remover)]
df_filtrado

,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,trava_zap,text_content_anonymous,dataset_info_id,date_system,score_sentiment,score_misinformation,id_message,id_persona,message_type,messenger,media_name,media_md5
0,2022-10-10 18:20:24,9d737b3c9387855139bbad2311cc5709,0638569ee76dac58f59dcac20463c955,NaN,NaN,NaN,False,False,False,☝️\nHoje que eu fique sabendo do CANAL LULA FL...,5,2022-10-10 18:20:25.000937,-0.7003,0.843775,F7023FFB06C429A2C166922849A35ED8,558594228826.0:12@s.whatsapp.net,Texto,whatsapp,NaN,NaN
1,2022-10-10 22:02:58,1660a60f661754d2802ca53296e25be8,a5910d5cc1c830ade9eb4dd00f15ff6a,NaN,NaN,NaN,False,False,False,Mais pra que isso não aconteça nois temos quê ...,5,2022-10-10 22:02:58.4682,-0.2960,NaN,A9FAC78070C144890D181EF415B90CAD,558594228826.0:12@s.whatsapp.net,TextoExtendido,whatsapp,NaN,NaN
2,2022-10-11 00:39:31,c882172d447798d74915973ac83eba68,b84dfe2d1599b82768dcdecce7e6bb23,d2e0ec59ffd9f84764f5b147725d7196.oga,audio/ogg; codecs=opus,NaN,True,False,False,NaN,5,2022-10-11 00:39:33.445125,NaN,NaN,737948BE86D450A426470794F91BC80D,558594228826.0:12@s.whatsapp.net,Audio,whatsapp,NaN,d2e0ec59ffd9f84764f5b147725d7196
3,2022-10-10 23:36:19,91e3c22c08b24ba01ac4524d77bcb1da,addb88a34374d43aa9ecd4df7359ce39,NaN,NaN,NaN,False,False,False,‎Acesse este link para entrar no meu grupo do ...,5,2022-10-10 23:36:19.724987,-0.1531,NaN,439A91ADD8F355CD23C4BB107A5E88BB,558594228826.0:12@s.whatsapp.net,TextoExtendido,whatsapp,NaN,NaN
4,2022-10-10 23:40:12,77c1a8a31fee269db258a028a61f0b88,a5831b109d4d95fae8ee0ce464e48a6a,bb80cd530332bb6b95c34e719992d249.f4v,video/mp4,NaN,True,False,False,NaN,5,2022-10-10 23:40:14.397495,NaN,NaN,092203A082AC3DFB2A4933F60453AEB8,558594228826.0:12@s.whatsapp.net,Video,whatsapp,NaN,bb80cd530332bb6b95c34e719992d249
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598966,2022-11-13 00:46:56,b1b419f54664f6cd4fcdb40425862d36,b1b419f54664f6cd4fcdb40425862d36,7719ba57cbe74eb5c8059d6b1a14d1ca.f4v,video/mp4,NaN,True,False,False,NaN,5,2023-01-30 18:20:22.262572,NaN,NaN,08E682716448AB06DEE126A30E7AAA77,558594228826.0:17@s.whatsapp.net,Video,whatsapp,NaN,7719ba57cbe74eb5c8059d6b1a14d1ca
598967,2022-11-13 13:37:03,2ad743e9ff831a75911a983b42205845,64c9a7c8ac338faa258e5aea9a1c2ae3,c6d59f0b86e63dbcce7fc682a1eaea50.f4v,video/mp4,NaN,True,False,False,NaN,5,2023-01-30 18:20:22.269632,NaN,NaN,3EB0259ED9B8FACF18A9,558594228826.0:17@s.whatsapp.net,Video,whatsapp,NaN,c6d59f0b86e63dbcce7fc682a1eaea50
598968,2022-11-13 17:43:22,b1b419f54664f6cd4fcdb40425862d36,b1b419f54664f6cd4fcdb40425862d36,003ace942d9665adc9647df9d9526c0c.jpeg,image/jpeg,NaN,True,False,False,NaN,5,2023-01-30 18:20:22.277283,NaN,NaN,5CD9F23CD07BEE38693E68E11F9C6A55,558594228826.0:17@s.whatsapp.net,Imagem,whatsapp,NaN,003ace942d9665adc9647df9d9526c0c
598969,2022-11-13 21:40:47,b1b419f54664f6cd4fcdb40425862d36,b1b419f54664f6cd4fcdb40425862d36,bab5653a94b097404739a061dbbc7407.f4v,video/mp4,NaN,True,False,False,NaN,5,2023-01-30 18:20:22.284021,NaN,NaN,D3BDE4A5C68E7148C0925F6C582C99A9,558594228826.0:17@s.whatsapp.net,Video,whatsapp,NaN,bab5653a94b097404739a061dbbc7407


In [7]:
# Removendo linhas com valores NaN na coluna 'text_content_anonymous'
df_filtrado = df_filtrado.dropna(subset=['text_content_anonymous'])

# Removendo trava_zap
df_filtrado = df_filtrado[df_filtrado['trava_zap'] == False]



#df_filtrado = df_filtrado[0:100]
len(df_filtrado)

254480

In [8]:

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('portuguese'))

custom_stopwords = {
    't', 'be', 'nao', 'youtu', 'vai', 'av', 'ja', 'to', 'the', 'this', 'i', 'and', 
    'you', 'y', 'www', 'sao', 'pois', 'contra', 'user', 'ai', 'so', 'gente', 'voce', 'of', 
    'ola', 'gift', 'card', 'kwaivideo', 'r', 'bom', 'q', 'vc', 'vcs', 'pra','ta', 'phone', 'ok', 'la',
    'sera', 'ha', 'aqui', 'ate', 'dia', 'mc', 'im', 'tmj', 'pix', 'g', 'diz', 'ti', 'etc', 'tudo', 
    'todo', 'toda', 'youtube', 'g1', 'm', 'instagram', 'fb', 'in', 'link', 'was', 'blocked', 'kk'
}

stop_words.update(custom_stopwords)

def preprocess_text(text):

    # Função para extrair e substituir o domínio da URL
    def substituir_dominios(texto):
        # Função para extrair e substituir o domínio da URL
        def extrair_dominio(url):
            # Remove o protocolo (http://, https://, etc.) e o "www." se presente
            dominio = re.sub(r'^https?://(?:www\.)?|www\.', '', url)
            # Remove o caminho e parâmetros da URL
            dominio = re.split(r'[/?#]', dominio)[0]
            # Retorna a parte principal do domínio (antes do primeiro ponto)
            return dominio.split('.')[0]

        # Substitui URLs por seus domínios principais
        return re.sub(r'https?://(?:www\.)?\S+|www\.\S+', lambda match: extrair_dominio(match.group(0)), texto)

    # Substituir domínios
    text = substituir_dominios(text)

    # Converte para minúsculas
    text = text.lower()

    # Remove acentos
    text = unidecode(text)

    #Remover Pontuação
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove URLs e menções
    #text = re.sub(r'http\S+|www\S+|https\S+|@\w+', '', text)

    # Substitui emojis repetidos por apenas um
    text = re.sub(r'([\U00010000-\U0010FFFF])\1+', r'\1', text)
    text = re.sub(r'([\U0001F600-\U0001F64F]|[\U0001F300-\U0001F5FF]|[\U0001F680-\U0001F6FF]|[\U0001F700-\U0001F77F]|[\U0001F780-\U0001F7FF]|[\U0001F800-\U0001F8FF]|[\U0001F900-\U0001F9FF]|[\U0001FA00-\U0001FA6F]|[\U0001FA70-\U0001FAFF])\1+', r'\1', text)


    # Remove espaços em branco extras (início ou final) e múltiplos espaços no meio do texto
    text = re.sub(r'\s+', ' ', text).strip()

    # Remove pontuações e caracteres especiais
    #text = re.sub(r'[^\w\s]', '', text)

    # Ajusta risadas "kkk" ou mais para "kk"
    text = re.sub(r'k{2,}|K{2,}', 'kk', text)

    # Ajusta risadas "haha" ou mais para "haha"
    text = re.sub(r'(ha){2,}', 'haha', text, flags=re.IGNORECASE)

    # Ajusta risadas "kaka" ou mais para "kaka"
    text = re.sub(r'(ka){2,}', 'kaka', text, flags=re.IGNORECASE)

    # Remove as stopwords
    text = ' '.join([word for word in text.split() if word not in stop_words])


    return text


[nltk_data] Downloading package punkt to C:\Users\Melissa
[nltk_data]     Felipe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Melissa
[nltk_data]     Felipe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Melissa
[nltk_data]     Felipe\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [9]:

df_geral = df_filtrado.copy()

# Aplicar o pré-processamento à coluna de texto
df_geral['text_processed'] = df_geral['text_content_anonymous'].apply(preprocess_text)
df_geral.head()

,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,trava_zap,text_content_anonymous,...,date_system,score_sentiment,score_misinformation,id_message,id_persona,message_type,messenger,media_name,media_md5,text_processed
0,2022-10-10 18:20:24,9d737b3c9387855139bbad2311cc5709,0638569ee76dac58f59dcac20463c955,NaN,NaN,NaN,False,False,False,☝️\nHoje que eu fique sabendo do CANAL LULA FL...,...,2022-10-10 18:20:25.000937,-0.7003,0.843775,F7023FFB06C429A2C166922849A35ED8,558594228826.0:12@s.whatsapp.net,Texto,whatsapp,NaN,NaN,hoje fique sabendo canal lula flix vi pt entro...
1,2022-10-10 22:02:58,1660a60f661754d2802ca53296e25be8,a5910d5cc1c830ade9eb4dd00f15ff6a,NaN,NaN,NaN,False,False,False,Mais pra que isso não aconteça nois temos quê ...,...,2022-10-10 22:02:58.4682,-0.2960,NaN,A9FAC78070C144890D181EF415B90CAD,558594228826.0:12@s.whatsapp.net,TextoExtendido,whatsapp,NaN,NaN,aconteca nois fazer anossa parte
3,2022-10-10 23:36:19,91e3c22c08b24ba01ac4524d77bcb1da,addb88a34374d43aa9ecd4df7359ce39,NaN,NaN,NaN,False,False,False,‎Acesse este link para entrar no meu grupo do ...,...,2022-10-10 23:36:19.724987,-0.1531,NaN,439A91ADD8F355CD23C4BB107A5E88BB,558594228826.0:12@s.whatsapp.net,TextoExtendido,whatsapp,NaN,NaN,acesse entrar grupo whatsapp chat
7,2022-10-10 16:49:48,325720ed3339a91b8076df12c1e95c45,0e345813dcb62b0fe4d8537f311af0f1,NaN,NaN,https://m.kwai.com/photo/150000006567403/52240...,False,True,False,Fortes palavras da Pastora Damares!\nhttps://k...,...,2022-10-10 16:49:50.051126,0.0000,NaN,F18BADED5AFA8FB0C33FE36625872DB8,558594228826.0:12@s.whatsapp.net,TextoExtendido,whatsapp,NaN,NaN,fortes palavras pastora damares
8,2022-10-10 16:49:53,8ed44a70a011285622e2b8919c2c8c3e,3b3e64e81c3e3f3bfc5ba76ddb90fb2d,0d6f2896e5941ef1933e69bbd7a32f69.jpeg,image/jpeg,NaN,True,False,False,🤣🤣🤣🤣🤣🤣🤣🤣,...,2022-10-10 16:49:54.713831,0.0000,NaN,724C233591C3A62A5231B47FF22ADE4B,558594228826.0:12@s.whatsapp.net,Imagem,whatsapp,NaN,0d6f2896e5941ef1933e69bbd7a32f69,


### Calculando a diversidade dos tópicos
A diversidade de tópicos mede quantas palavras únicas existem entre os top-N termos de todos os tópicos. 

In [10]:
def topic_diversity(model, top_n=10):
    topic_words = []
    for k in range(model.k):  
        words = [word for word, _ in model.get_topic_words(k, top_n=top_n)]  
        topic_words.extend(words)
    unique_words = set(topic_words)
    return len(unique_words) / (top_n * model.k)


### IRBO 

IRBO avalia quanto os tópicos são distintos entre si, usando o Rank-Biased Overlap (RBO) invertido. Quanto maior o IRBO, mais diferentes são os tópicos

In [11]:
import rbo

def compute_irbo_ptm(model, top_n=10):
    topics = []
    for k in range(model.k):
        topic_terms = [word for word, _ in model.get_topic_words(k, top_n=top_n)]
        topics.append(topic_terms)

    n = len(topics)
    total_irbo = 0
    count = 0

    for i in range(n):
        for j in range(i + 1, n):
            rbo_score = rbo.RankingSimilarity(topics[i], topics[j]).rbo()
            total_irbo += (1 - rbo_score)  # IRBO = 1 - RBO
            count += 1

    return total_irbo / count if count > 0 else 0


In [12]:
# Processamento
token_espaco = tokenize.WhitespaceTokenizer()
token_pontuacao = tokenize.WordPunctTokenizer()

In [13]:
def compute_coherence_ptm(texts, id2word, coherence, start=2, limit=30, step=2):
    coherence_values = []
    model_list = []
    topic_range = list(range(start, limit, step))

    for num_topics in tqdm(topic_range, desc=f"Treinando PTModel ({coherence})"):
        model = tp.PTModel(k=num_topics, seed=42)
        for doc in texts:
            model.add_doc(doc)
        model.train(100)
        model_list.append(model)

        # Coherence do gensim precisa de tópicos no formato: lista de listas de palavras
        topics = []
        for k in range(model.k):
            topic_words = [word for word, _ in model.get_topic_words(k, top_n=10)]
            topics.append(topic_words)

        coh_model = CoherenceModel(topics=topics, texts=texts, dictionary=id2word, coherence=coherence)
        coherence_score = coh_model.get_coherence()
        coherence_values.append(coherence_score)

    optimal_index = coherence_values.index(max(coherence_values))
    optimal_num_topics = topic_range[optimal_index]

    return optimal_num_topics, model_list, coherence_values


In [14]:
def processar_texto(texto):
    texto = texto.lower()
    palavras_texto = token_espaco.tokenize(texto)
    palavras_texto = token_pontuacao.tokenize(' '.join(palavras_texto))
    return [palavra for palavra in palavras_texto]


## Modelando os Tópicos sem filtro 

In [15]:
texto_processado = df_geral['text_processed'].apply(processar_texto)

# Corpus e Dicionário (para CoherenceModel do gensim)
id2word = corpora.Dictionary(texto_processado)
corpus = [id2word.doc2bow(text) for text in texto_processado]
texts = texto_processado.tolist()

In [16]:
def calculate_ptm_coherence_for_metrics(texts, id2word, metrics, start=2, limit=30, step=2):
    results = {}
    for metric in metrics:
        num_topics, model_list, coherence_values = compute_coherence_ptm(
            texts=texts,
            id2word=id2word,
            coherence=metric,
            start=start,
            limit=limit,
            step=step
        )
        results[metric] = {
            'num_topics': num_topics,
            'model_list': model_list,
            'coherence_values': coherence_values
        }
    return results

metrics = ['c_v', 'u_mass', 'c_uci', 'c_npmi']

ptm_coherence_results = calculate_ptm_coherence_for_metrics(
    texts=texto_processado.tolist(),
    id2word=id2word,
    metrics=metrics,
    start=2,
    limit=30,
    step=2
)


Treinando PTModel (c_v):   0%|          | 0/14 [00:00<?, ?it/s]C:\Users\Melissa Felipe\AppData\Local\Temp\ipykernel_26484\2436467606.py:10: RuntimeWarning: The training result may differ even with fixed seed if `workers` != 1.
  model.train(100)
Treinando PTModel (c_npmi): 100%|██████████| 14/14 [33:42<00:00, 144.47s/it]


In [ ]:
# Mostrar resultados
for metric in metrics:
    print(f"\nResultados de Coerência para {metric.upper()}:")
    for m, cv in zip(range(2, 30, 2), ptm_coherence_results[metric]['coherence_values']):
        print(f"Qtd. tópicos = {m} -> Coerência = {round(cv, 4)}")
    print(f"Melhor número de tópicos segundo {metric.upper()}: {ptm_coherence_results[metric]['num_topics']}")



Resultados de Coerência para C_V:
Qtd. tópicos = 2 -> Coerência = 0.4838
Qtd. tópicos = 4 -> Coerência = 0.4564
Qtd. tópicos = 6 -> Coerência = 0.4733
Qtd. tópicos = 8 -> Coerência = 0.5115
Qtd. tópicos = 10 -> Coerência = 0.5222
Qtd. tópicos = 12 -> Coerência = 0.511
Qtd. tópicos = 14 -> Coerência = 0.5195
Qtd. tópicos = 16 -> Coerência = 0.5575
Qtd. tópicos = 18 -> Coerência = 0.4994
Qtd. tópicos = 20 -> Coerência = 0.5579
Qtd. tópicos = 22 -> Coerência = 0.548
Qtd. tópicos = 24 -> Coerência = 0.5455
Qtd. tópicos = 26 -> Coerência = 0.5716
Qtd. tópicos = 28 -> Coerência = 0.5514
Melhor número de tópicos segundo C_V: 26

Resultados de Coerência para U_MASS:
Qtd. tópicos = 2 -> Coerência = -5.1244
Qtd. tópicos = 4 -> Coerência = -7.8644
Qtd. tópicos = 6 -> Coerência = -5.4186
Qtd. tópicos = 8 -> Coerência = -4.7261
Qtd. tópicos = 10 -> Coerência = -5.14
Qtd. tópicos = 12 -> Coerência = -5.0656
Qtd. tópicos = 14 -> Coerência = -4.6248
Qtd. tópicos = 16 -> Coerência = -3.907
Qtd. tópico

In [ ]:
# Escolha da métrica
chosen_metric = 'c_npmi'  # ou 'c_v', 'u_mass', 'c_uci'

# Obter o melhor modelo treinado
best_index = ptm_coherence_results[chosen_metric]['coherence_values'].index(
    max(ptm_coherence_results[chosen_metric]['coherence_values'])
)
best_model = ptm_coherence_results[chosen_metric]['model_list'][best_index]

# Gerar DataFrame com tópicos
def get_topics_dataframe(model, top_n=10):
    topics_data = []
    for topic_id in range(model.k):
        words_probs = model.get_topic_words(topic_id, top_n=top_n)
        topic_words = [word for word, prob in words_probs]
        topic_probs = [prob for word, prob in words_probs]
        topics_data.append({
            'Tópico': topic_id,
            'Palavras': ', '.join(topic_words),
            'Pesos': topic_probs
        })
    return pd.DataFrame(topics_data)

df_topicos = get_topics_dataframe(best_model)

df_topicos.to_csv('./resultados_pseudo/wpp/pseudo_wpp_c_npmi.csv', index=False)
df_topicos.head(10)

,Tópico,Palavras,Pesos
0,0,"99999999999999, excluir, studio, grupo, adm, b...","[0.7779877185821533, 0.02684282884001732, 0.02..."
1,1,"lula, bolsonaro, moraes, todos, alexandre, vol...","[0.016064640134572983, 0.012980114668607712, 0..."
2,2,"pag, pt, emprestimo, item, todos, entrada, 160...","[0.011205372400581837, 0.010022246278822422, 0..."
3,3,"bocahomens, cagando, bosta, carae, vomitando, ...","[0.18525643646717072, 0.13916648924350739, 0.1..."
4,4,"bolsonaro, todos, povo, estados, presidente, r...","[0.010354984551668167, 0.007956349290907383, 0..."
5,5,"lula, proibido, dizer, tse, sobre, pt, stf, br...","[0.17831216752529144, 0.0439273975789547, 0.03..."
6,6,"55, 20, 1, 91, 92, 127, 155, 111, 128, 101","[0.23404990136623383, 0.062362492084503174, 0...."
7,7,"terrabrasilnoticias, stf, estao, presidente, p...","[0.008691192604601383, 0.00766870379447937, 0...."
8,8,"8l8l8l8l8l8l8l8l8l8lli, brasil, fez, revolta, ...","[0.013670983724296093, 0.011046798899769783, 0..."
9,9,"gazetabrasil, anos, noticias, 2022, brasil, ap...","[0.012686355970799923, 0.00932218600064516, 0...."


In [19]:
div = topic_diversity(best_model)
print(f"Diversidade de Tópicos: {round(div, 4)}")

irbo_score = compute_irbo_ptm(best_model)
print(f"IRBO médio entre tópicos: {round(irbo_score, 4)}")


Diversidade de Tópicos: 0.7808
IRBO médio entre tópicos: 0.9591


In [ ]:
# Escolha da métrica
chosen_metric = 'c_v'  # ou 'c_v', 'u_mass', 'c_uci'

# Obter o melhor modelo treinado
best_index = ptm_coherence_results[chosen_metric]['coherence_values'].index(
    max(ptm_coherence_results[chosen_metric]['coherence_values'])
)
best_model = ptm_coherence_results[chosen_metric]['model_list'][best_index]

# Gerar DataFrame com tópicos
def get_topics_dataframe(model, top_n=10):
    topics_data = []
    for topic_id in range(model.k):
        words_probs = model.get_topic_words(topic_id, top_n=top_n)
        topic_words = [word for word, prob in words_probs]
        topic_probs = [prob for word, prob in words_probs]
        topics_data.append({
            'Tópico': topic_id,
            'Palavras': ', '.join(topic_words),
            'Pesos': topic_probs
        })
    return pd.DataFrame(topics_data)

df_topicos = get_topics_dataframe(best_model)

df_topicos.to_csv('./resultados_pseudo/wpp/pseudo_wpp_c_v.csv', index=False)
df_topicos.head(10)

,Tópico,Palavras,Pesos
0,0,"99999999999999, excluir, studio, grupo, adm, b...","[0.7779877185821533, 0.02684282884001732, 0.02..."
1,1,"lula, bolsonaro, moraes, todos, alexandre, vol...","[0.016064640134572983, 0.012980114668607712, 0..."
2,2,"pag, pt, emprestimo, item, todos, entrada, 160...","[0.011205372400581837, 0.010022246278822422, 0..."
3,3,"bocahomens, cagando, bosta, carae, vomitando, ...","[0.18525643646717072, 0.13916648924350739, 0.1..."
4,4,"bolsonaro, todos, povo, estados, presidente, r...","[0.010354984551668167, 0.007956349290907383, 0..."
5,5,"lula, proibido, dizer, tse, sobre, pt, stf, br...","[0.17831216752529144, 0.0439273975789547, 0.03..."
6,6,"55, 20, 1, 91, 92, 127, 155, 111, 128, 101","[0.23404990136623383, 0.062362492084503174, 0...."
7,7,"terrabrasilnoticias, stf, estao, presidente, p...","[0.008691192604601383, 0.00766870379447937, 0...."
8,8,"8l8l8l8l8l8l8l8l8l8lli, brasil, fez, revolta, ...","[0.013670983724296093, 0.011046798899769783, 0..."
9,9,"gazetabrasil, anos, noticias, 2022, brasil, ap...","[0.012686355970799923, 0.00932218600064516, 0...."


In [21]:
div = topic_diversity(best_model)
print(f"Diversidade de Tópicos: {round(div, 4)}")

irbo_score = compute_irbo_ptm(best_model)
print(f"IRBO médio entre tópicos: {round(irbo_score, 4)}")


Diversidade de Tópicos: 0.7808
IRBO médio entre tópicos: 0.9591


In [ ]:
# Escolha da métrica
chosen_metric = 'c_uci'  # ou 'c_v', 'u_mass', 'c_uci'

# Obter o melhor modelo treinado
best_index = ptm_coherence_results[chosen_metric]['coherence_values'].index(
    max(ptm_coherence_results[chosen_metric]['coherence_values'])
)
best_model = ptm_coherence_results[chosen_metric]['model_list'][best_index]

# Gerar DataFrame com tópicos
def get_topics_dataframe(model, top_n=10):
    topics_data = []
    for topic_id in range(model.k):
        words_probs = model.get_topic_words(topic_id, top_n=top_n)
        topic_words = [word for word, prob in words_probs]
        topic_probs = [prob for word, prob in words_probs]
        topics_data.append({
            'Tópico': topic_id,
            'Palavras': ', '.join(topic_words),
            'Pesos': topic_probs
        })
    return pd.DataFrame(topics_data)

df_topicos = get_topics_dataframe(best_model)

df_topicos.to_csv('./resultados_pseudo/wpp/pseudo_wpp_c_uci.csv', index=False)
df_topicos.head(10)

,Tópico,Palavras,Pesos
0,0,"lula, vamos, brasil, bolsonaro, povo, presiden...","[0.01962336152791977, 0.013131478801369667, 0...."
1,1,"joao, saymon, homessexual, divirtace, km, br, ...","[0.05603599548339844, 0.046925708651542664, 0...."
2,2,"99999999999999, r, wr273ymrnwbddyrblwch, ezgz,...","[0.9532402753829956, 0.0008700666367076337, 0...."
3,3,"grupos, silva, 11, brasil, bolsonaro, 14, 24, ...","[0.01112956553697586, 0.010227174498140812, 0...."
4,4,"bocahomens, cagando, bosta, carae, vomitando, ...","[0.182708740234375, 0.13762100040912628, 0.107..."
5,5,"milhoes, caso, escandalo, pt, bilhoes, 2018, l...","[0.0235182773321867, 0.01338708121329546, 0.01..."
6,6,"brasil, 2022, portaltocanews, anos, mil, sobre...","[0.007906096056103706, 0.006553787738084793, 0..."
7,7,"lula, volta, todos, eleicoes, bolsonaro, brasi...","[0.012261259369552135, 0.011130464263260365, 0..."
8,8,"janja, robo, proibido, dizer, lula, stf, tse, ...","[0.09918999671936035, 0.09720946848392487, 0.0..."
9,9,"sim, voto, 22, porque, concorda, grupo, pais, ...","[0.018109342083334923, 0.012574584223330021, 0..."


In [23]:
div = topic_diversity(best_model)
print(f"Diversidade de Tópicos: {round(div, 4)}")

irbo_score = compute_irbo_ptm(best_model)
print(f"IRBO médio entre tópicos: {round(irbo_score, 4)}")


Diversidade de Tópicos: 0.77
IRBO médio entre tópicos: 0.9496


In [ ]:
# Escolha da métrica
chosen_metric = 'u_mass'  # ou 'c_v', 'u_mass', 'c_uci'

# Obter o melhor modelo treinado
best_index = ptm_coherence_results[chosen_metric]['coherence_values'].index(
    max(ptm_coherence_results[chosen_metric]['coherence_values'])
)
best_model = ptm_coherence_results[chosen_metric]['model_list'][best_index]

# Gerar DataFrame com tópicos
def get_topics_dataframe(model, top_n=10):
    topics_data = []
    for topic_id in range(model.k):
        words_probs = model.get_topic_words(topic_id, top_n=top_n)
        topic_words = [word for word, prob in words_probs]
        topic_probs = [prob for word, prob in words_probs]
        topics_data.append({
            'Tópico': topic_id,
            'Palavras': ', '.join(topic_words),
            'Pesos': topic_probs
        })
    return pd.DataFrame(topics_data)

df_topicos = get_topics_dataframe(best_model)

df_topicos.to_csv('./resultados_pseudo/wpp/pseudo_wpp_u_mass.csv', index=False)
df_topicos.head(10)

,Tópico,Palavras,Pesos
0,0,"stf, moraes, alexandre, qualquer, direito, min...","[0.011127772741019726, 0.01059278566390276, 0...."
1,1,"divirtace, internet, 25, 2, valor, fortaleza, ...","[0.009107273072004318, 0.005588032305240631, 0..."
2,2,"55, 20, 1, 91, 92, 127, 155, 115, 128, 101","[0.149518683552742, 0.10145100951194763, 0.043..."
3,3,"bolsonaro, brasil, rio, estados, 3, pesquisa, ...","[0.03183000162243843, 0.019425008445978165, 0...."
4,4,"bocahomens, cagando, bosta, carae, vomitando, ...","[0.18141232430934906, 0.1366688758134842, 0.11..."
5,5,"gazetabrasil, brasil, noticias, grupo, apos, a...","[0.014604044146835804, 0.013515135273337364, 0..."
6,6,"lula, escandalo, milhoes, habitantes, votaram,...","[0.04103682562708855, 0.021463025361299515, 0...."
7,7,"fazendo, olha, tao, covardia, pronuncie, franc...","[0.03546610847115517, 0.035082414746284485, 0...."
8,8,"lula, bolsonaro, pt, brasil, votos, tse, video...","[0.06241956725716591, 0.041065555065870285, 0...."
9,9,"eleicoes, bolsonaro, porque, urnas, tse, turno...","[0.013579479418694973, 0.010746631771326065, 0..."


In [25]:
div = topic_diversity(best_model)
print(f"Diversidade de Tópicos: {round(div, 4)}")

irbo_score = compute_irbo_ptm(best_model)
print(f"IRBO médio entre tópicos: {round(irbo_score, 4)}")


Diversidade de Tópicos: 0.7458
IRBO médio entre tópicos: 0.9585


## Modelagem de tópicos com filtro religioso

In [26]:
palavras_religiosas = [
    "deus", "jesus", "misericordia", "davi",
    "salomao", "reino", "templo", "conservador",
    "pentecostal", "rcc", "renovacao", "carismatic",
    "paulo ricardo", "bernardo kuster", "herege", "ateu",
    "jerico", "heresia"
]


In [27]:
# Função para verificar se uma palavra está relacionada à religião
def relacionada_religiao(word):
    word_lower = word.lower()
    palavras_religiosas_lower = [palavra.lower() for palavra in palavras_religiosas]

    # Verificando se alguma palavra da lista de palavras religiosas está presente
    return any(palavra in word_lower for palavra in palavras_religiosas_lower)


df_religiao = df_geral[df_geral['text_processed'].apply(lambda x: relacionada_religiao(x))]



In [28]:
texto_processado_religiao = df_religiao['text_processed'].apply(processar_texto)

In [29]:
# Corpus e Dicionário (para CoherenceModel do gensim)
id2word_religiao = corpora.Dictionary(texto_processado_religiao)
corpus_religiao = [id2word.doc2bow(text) for text in texto_processado_religiao]
texts_religiao = texto_processado_religiao.tolist()

In [30]:
def calculate_ptm_coherence_for_metrics(texts_religiao, id2word_religiao, metrics, start=2, limit=30, step=2):
    results = {}
    for metric in metrics:
        num_topics, model_list, coherence_values = compute_coherence_ptm(
            texts=texts_religiao,
            id2word=id2word_religiao,
            coherence=metric,
            start=start,
            limit=limit,
            step=step
        )
        results[metric] = {
            'num_topics': num_topics,
            'model_list': model_list,
            'coherence_values': coherence_values
        }
    return results

metrics = ['c_v', 'u_mass', 'c_uci', 'c_npmi']

ptm_coherence_results = calculate_ptm_coherence_for_metrics(
    texts_religiao=texto_processado_religiao.tolist(),
    id2word_religiao=id2word_religiao,
    metrics=metrics,
    start=2,
    limit=30,
    step=2
)


Treinando PTModel (c_v):   0%|          | 0/14 [00:00<?, ?it/s]C:\Users\Melissa Felipe\AppData\Local\Temp\ipykernel_26484\2436467606.py:10: RuntimeWarning: The training result may differ even with fixed seed if `workers` != 1.
  model.train(100)
Treinando PTModel (c_npmi): 100%|██████████| 14/14 [04:39<00:00, 19.97s/it]


In [ ]:
# Mostrar resultados
for metric in metrics:
    print(f"\nResultados de Coerência para {metric.upper()}:")
    for m, cv in zip(range(2, 30, 2), ptm_coherence_results[metric]['coherence_values']):
        print(f"Qtd. tópicos = {m} -> Coerência = {round(cv, 4)}")
    print(f"Melhor número de tópicos segundo {metric.upper()}: {ptm_coherence_results[metric]['num_topics']}")



Resultados de Coerência para C_V:
Qtd. tópicos = 2 -> Coerência = 0.4559
Qtd. tópicos = 4 -> Coerência = 0.5554
Qtd. tópicos = 6 -> Coerência = 0.5611
Qtd. tópicos = 8 -> Coerência = 0.6496
Qtd. tópicos = 10 -> Coerência = 0.5808
Qtd. tópicos = 12 -> Coerência = 0.541
Qtd. tópicos = 14 -> Coerência = 0.6015
Qtd. tópicos = 16 -> Coerência = 0.5493
Qtd. tópicos = 18 -> Coerência = 0.5622
Qtd. tópicos = 20 -> Coerência = 0.5785
Qtd. tópicos = 22 -> Coerência = 0.6058
Qtd. tópicos = 24 -> Coerência = 0.5712
Qtd. tópicos = 26 -> Coerência = 0.5965
Qtd. tópicos = 28 -> Coerência = 0.6057
Melhor número de tópicos segundo C_V: 8

Resultados de Coerência para U_MASS:
Qtd. tópicos = 2 -> Coerência = -3.8858
Qtd. tópicos = 4 -> Coerência = -1.6734
Qtd. tópicos = 6 -> Coerência = -1.8377
Qtd. tópicos = 8 -> Coerência = -1.7746
Qtd. tópicos = 10 -> Coerência = -1.5814
Qtd. tópicos = 12 -> Coerência = -1.7741
Qtd. tópicos = 14 -> Coerência = -1.6836
Qtd. tópicos = 16 -> Coerência = -1.7514
Qtd. tóp

In [ ]:
# Escolha da métrica
chosen_metric = 'c_npmi'  # ou 'c_v', 'u_mass', 'c_uci'

# Obter o melhor modelo treinado
best_index = ptm_coherence_results[chosen_metric]['coherence_values'].index(
    max(ptm_coherence_results[chosen_metric]['coherence_values'])
)
best_model = ptm_coherence_results[chosen_metric]['model_list'][best_index]

# Gerar DataFrame com tópicos
def get_topics_dataframe(model, top_n=10):
    topics_data = []
    for topic_id in range(model.k):
        words_probs = model.get_topic_words(topic_id, top_n=top_n)
        topic_words = [word for word, prob in words_probs]
        topic_probs = [prob for word, prob in words_probs]
        topics_data.append({
            'Tópico': topic_id,
            'Palavras': ', '.join(topic_words),
            'Pesos': topic_probs
        })
    return pd.DataFrame(topics_data)

df_topicos = get_topics_dataframe(best_model)

df_topicos.to_csv('./resultados_pseudo/wpp/pseudo_wpp_religioso_c_npmi.csv', index=False)

div = topic_diversity(best_model)
print(f"Diversidade de Tópicos: {round(div, 4)}")

irbo_score = compute_irbo_ptm(best_model)
print(f"IRBO médio entre tópicos: {round(irbo_score, 4)}")

df_topicos.head(10)

Diversidade de Tópicos: 0.7036
IRBO médio entre tópicos: 0.9516


,Tópico,Palavras,Pesos
0,0,"drive, brasil, bolsonaro, povo, todos, video, ...","[0.020388666540384293, 0.010153494775295258, 0..."
1,1,"lula, urnas, eleicoes, bolsonaro, vitoria, pt,...","[0.021928688511252403, 0.020492324605584145, 0..."
2,2,"pessoas, mil, errou, gazetabrasil, milhoes, qu...","[0.01068599708378315, 0.00648313295096159, 0.0..."
3,3,"3, 2, 1, 10, 5, brasil, 15, 22, 8, 16","[0.02451811544597149, 0.023244017735123634, 0...."
4,4,"turno, alexandre, arruda, roberto, assim, jose...","[0.01781422831118107, 0.013747712597250938, 0...."
5,5,"direito, pode, manifestacao, todos, caso, enta...","[0.01690850220620632, 0.01619306020438671, 0.0..."
6,6,"fez, brasil, ferrovia, br, deu, coisas, agua, ...","[0.025215422734618187, 0.021936312317848206, 0..."
7,7,"tribunal, republica, stf, supremo, geral, acao...","[0.025822211056947708, 0.019148360937833786, 0..."
8,8,"lula, brasil, pt, pode, governo, sobre, defend...","[0.07110974192619324, 0.03516508638858795, 0.0..."
9,9,"presidente, liberdade, todos, patria, direito,...","[0.03478185087442398, 0.0267342459410429, 0.02..."


In [ ]:
# Escolha da métrica
chosen_metric = 'c_v'  # ou 'c_v', 'u_mass', 'c_uci'

# Obter o melhor modelo treinado
best_index = ptm_coherence_results[chosen_metric]['coherence_values'].index(
    max(ptm_coherence_results[chosen_metric]['coherence_values'])
)
best_model = ptm_coherence_results[chosen_metric]['model_list'][best_index]

# Gerar DataFrame com tópicos
def get_topics_dataframe(model, top_n=10):
    topics_data = []
    for topic_id in range(model.k):
        words_probs = model.get_topic_words(topic_id, top_n=top_n)
        topic_words = [word for word, prob in words_probs]
        topic_probs = [prob for word, prob in words_probs]
        topics_data.append({
            'Tópico': topic_id,
            'Palavras': ', '.join(topic_words),
            'Pesos': topic_probs
        })
    return pd.DataFrame(topics_data)

df_topicos = get_topics_dataframe(best_model)

df_topicos.to_csv('./resultados_pseudo/wpp/pseudo_wpp_religioso_c_v.csv', index=False)

div = topic_diversity(best_model)
print(f"Diversidade de Tópicos: {round(div, 4)}")

irbo_score = compute_irbo_ptm(best_model)
print(f"IRBO médio entre tópicos: {round(irbo_score, 4)}")

df_topicos.head(10)

Diversidade de Tópicos: 0.7875
IRBO médio entre tópicos: 0.8939


,Tópico,Palavras,Pesos
0,0,"pl, bolsonaro, deputado, governador, 22, senad...","[0.03423726186156273, 0.015543557703495026, 0...."
1,1,"bolsonaro, grupos, 22, deus, brasil, grupo, ap...","[0.01913987658917904, 0.008895935490727425, 0...."
2,2,"lula, brasil, deus, bolsonaro, porque, preside...","[0.025525715202093124, 0.01919322833418846, 0...."
3,3,"deus, todos, senhor, jesus, bem, vida, povo, p...","[0.026477796956896782, 0.009984838776290417, 0..."
4,4,"55, 20, 1, 91, 92, 127, 155, 101, 128, 115","[0.2426958829164505, 0.06488315016031265, 0.03..."
5,5,"anos, brasil, 2022, noticias, apos, jrmunews, ...","[0.008514058776199818, 0.004735302180051804, 0..."
6,6,"brasil, bolsonaro, ok, todos, 22, fez, dizer, ...","[0.01728217490017414, 0.014552361331880093, 0...."
7,7,"eleicoes, lula, urnas, stf, turno, roberto, ts...","[0.013431498780846596, 0.013159469701349735, 0..."


In [ ]:
# Escolha da métrica
chosen_metric = 'u_mass'  # ou 'c_v', 'u_mass', 'c_uci'

# Obter o melhor modelo treinado
best_index = ptm_coherence_results[chosen_metric]['coherence_values'].index(
    max(ptm_coherence_results[chosen_metric]['coherence_values'])
)
best_model = ptm_coherence_results[chosen_metric]['model_list'][best_index]

# Gerar DataFrame com tópicos
def get_topics_dataframe(model, top_n=10):
    topics_data = []
    for topic_id in range(model.k):
        words_probs = model.get_topic_words(topic_id, top_n=top_n)
        topic_words = [word for word, prob in words_probs]
        topic_probs = [prob for word, prob in words_probs]
        topics_data.append({
            'Tópico': topic_id,
            'Palavras': ', '.join(topic_words),
            'Pesos': topic_probs
        })
    return pd.DataFrame(topics_data)

df_topicos = get_topics_dataframe(best_model)

df_topicos.to_csv('./resultados_pseudo/wpp/pseudo_wpp_religioso_u_mass.csv', index=False)

div = topic_diversity(best_model)
print(f"Diversidade de Tópicos: {round(div, 4)}")

irbo_score = compute_irbo_ptm(best_model)
print(f"IRBO médio entre tópicos: {round(irbo_score, 4)}")

df_topicos.head(10)

Diversidade de Tópicos: 0.7346
IRBO médio entre tópicos: 0.9485


,Tópico,Palavras,Pesos
0,0,"porque, entenderam, esquerda, pais, anos, demo...","[0.07183302938938141, 0.047818128019571304, 0...."
1,1,"estao, lei, dentro, algo, fogo, onde, nome, tr...","[0.006019809748977423, 0.00592648284509778, 0...."
2,2,"brasil, deus, bolsonaro, todos, presidente, ag...","[0.021332817152142525, 0.021316369995474815, 0..."
3,3,"noticias, pais, rio, apos, policia, sobre, pes...","[0.00921702291816473, 0.006201962009072304, 0...."
4,4,"coisas, agua, dinheiro, anos, 2, 3, 13, 10, 5,...","[0.018107086420059204, 0.01573595218360424, 0...."
5,5,"segundo, federal, brasil, milhoes, anos, brasi...","[0.015116135589778423, 0.015116135589778423, 0..."
6,6,"sr, eleicoes, alexandre, assim, roberto, turno...","[0.012554574757814407, 0.01106806006282568, 0...."
7,7,"lula, bolsonaro, estados, presidente, hoje, va...","[0.028039971366524696, 0.022211939096450806, 0..."
8,8,"agora, bem, pais, poder, militar, fazer, direi...","[0.016884535551071167, 0.013121739961206913, 0..."
9,9,"cand, brasileiro, brasil, enquanto, linha, lid...","[0.014604990370571613, 0.009701501578092575, 0..."


In [ ]:
# Escolha da métrica
chosen_metric = 'c_uci'  # ou 'c_v', 'u_mass', 'c_uci'

# Obter o melhor modelo treinado
best_index = ptm_coherence_results[chosen_metric]['coherence_values'].index(
    max(ptm_coherence_results[chosen_metric]['coherence_values'])
)
best_model = ptm_coherence_results[chosen_metric]['model_list'][best_index]

# Gerar DataFrame com tópicos
def get_topics_dataframe(model, top_n=10):
    topics_data = []
    for topic_id in range(model.k):
        words_probs = model.get_topic_words(topic_id, top_n=top_n)
        topic_words = [word for word, prob in words_probs]
        topic_probs = [prob for word, prob in words_probs]
        topics_data.append({
            'Tópico': topic_id,
            'Palavras': ', '.join(topic_words),
            'Pesos': topic_probs
        })
    return pd.DataFrame(topics_data)

df_topicos = get_topics_dataframe(best_model)

df_topicos.to_csv('./resultados_pseudo/wpp/pseudo_wpp_religioso_c_uci.csv', index=False)

div = topic_diversity(best_model)
print(f"Diversidade de Tópicos: {round(div, 4)}")

irbo_score = compute_irbo_ptm(best_model)
print(f"IRBO médio entre tópicos: {round(irbo_score, 4)}")

df_topicos.head(10)

Diversidade de Tópicos: 0.7036
IRBO médio entre tópicos: 0.9516


,Tópico,Palavras,Pesos
0,0,"drive, brasil, bolsonaro, povo, todos, video, ...","[0.020388666540384293, 0.010153494775295258, 0..."
1,1,"lula, urnas, eleicoes, bolsonaro, vitoria, pt,...","[0.021928688511252403, 0.020492324605584145, 0..."
2,2,"pessoas, mil, errou, gazetabrasil, milhoes, qu...","[0.01068599708378315, 0.00648313295096159, 0.0..."
3,3,"3, 2, 1, 10, 5, brasil, 15, 22, 8, 16","[0.02451811544597149, 0.023244017735123634, 0...."
4,4,"turno, alexandre, arruda, roberto, assim, jose...","[0.01781422831118107, 0.013747712597250938, 0...."
5,5,"direito, pode, manifestacao, todos, caso, enta...","[0.01690850220620632, 0.01619306020438671, 0.0..."
6,6,"fez, brasil, ferrovia, br, deu, coisas, agua, ...","[0.025215422734618187, 0.021936312317848206, 0..."
7,7,"tribunal, republica, stf, supremo, geral, acao...","[0.025822211056947708, 0.019148360937833786, 0..."
8,8,"lula, brasil, pt, pode, governo, sobre, defend...","[0.07110974192619324, 0.03516508638858795, 0.0..."
9,9,"presidente, liberdade, todos, patria, direito,...","[0.03478185087442398, 0.0267342459410429, 0.02..."


## Modelagem de tópicos com filtros religiosos e exclusão de político

In [36]:
palavras_politicas = [ "lula", "bolsonaro", "pt", "pl", "stf", "patria", "55", "22", "13", "senadores", "lulaladrao",
                       "urnas", "alexandre", "moraes", "comunismo", "eleicao", "eleicoes", "esquerda", "direita",
                         "presidente", "tse", "fraude", "voto", "turno", "ministro"]

In [37]:
def retirar_mensagens_com_palavras_politicas(word):
    word_lower = word.lower()
    palavras_politica_lower = [palavra.lower() for palavra in palavras_politicas]

    return any(palavra in word_lower for palavra in palavras_politica_lower)


df_politico = df_religiao[~df_religiao['text_processed'].apply(lambda x: retirar_mensagens_com_palavras_politicas(x))]



In [38]:
texto_processado_politico = df_politico['text_processed'].apply(processar_texto)

# Corpus e Dicionário (para CoherenceModel do gensim)
id2word_politico = corpora.Dictionary(texto_processado_politico)
corpus_politico = [id2word.doc2bow(text) for text in texto_processado_politico]
texts_politico = texto_processado_politico.tolist()

In [39]:
def calculate_ptm_coherence_for_metrics(texts_politico, id2word_politico, metrics, start=2, limit=30, step=2):
    results = {}
    for metric in metrics:
        num_topics, model_list, coherence_values = compute_coherence_ptm(
            texts=texts_politico,
            id2word=id2word_politico,
            coherence=metric,
            start=start,
            limit=limit,
            step=step
        )
        results[metric] = {
            'num_topics': num_topics,
            'model_list': model_list,
            'coherence_values': coherence_values
        }
    return results

metrics = ['c_v', 'u_mass', 'c_uci', 'c_npmi']

ptm_coherence_results = calculate_ptm_coherence_for_metrics(
    texts_politico=texto_processado_politico.tolist(),
    id2word_politico=id2word_politico,
    metrics=metrics,
    start=2,
    limit=30,
    step=2
)


Treinando PTModel (c_v):   0%|          | 0/14 [00:00<?, ?it/s]

C:\Users\Melissa Felipe\AppData\Local\Temp\ipykernel_26484\2436467606.py:10: RuntimeWarning: The training result may differ even with fixed seed if `workers` != 1.
  model.train(100)
Treinando PTModel (c_npmi): 100%|██████████| 14/14 [02:40<00:00, 11.43s/it]


In [ ]:

# Mostrar resultados
for metric in metrics:
    print(f"\nResultados de Coerência para {metric.upper()}:")
    for m, cv in zip(range(2, 30, 2), ptm_coherence_results[metric]['coherence_values']):
        print(f"Qtd. tópicos = {m} -> Coerência = {round(cv, 4)}")
    print(f"Melhor número de tópicos segundo {metric.upper()}: {ptm_coherence_results[metric]['num_topics']}")



Resultados de Coerência para C_V:
Qtd. tópicos = 2 -> Coerência = 0.6109
Qtd. tópicos = 4 -> Coerência = 0.4975
Qtd. tópicos = 6 -> Coerência = 0.4948
Qtd. tópicos = 8 -> Coerência = 0.5287
Qtd. tópicos = 10 -> Coerência = 0.5389
Qtd. tópicos = 12 -> Coerência = 0.4428
Qtd. tópicos = 14 -> Coerência = 0.5075
Qtd. tópicos = 16 -> Coerência = 0.5261
Qtd. tópicos = 18 -> Coerência = 0.4865
Qtd. tópicos = 20 -> Coerência = 0.5028
Qtd. tópicos = 22 -> Coerência = 0.4841
Qtd. tópicos = 24 -> Coerência = 0.5057
Qtd. tópicos = 26 -> Coerência = 0.5027
Qtd. tópicos = 28 -> Coerência = 0.4736
Melhor número de tópicos segundo C_V: 2

Resultados de Coerência para U_MASS:
Qtd. tópicos = 2 -> Coerência = -2.1423
Qtd. tópicos = 4 -> Coerência = -2.587
Qtd. tópicos = 6 -> Coerência = -3.9789
Qtd. tópicos = 8 -> Coerência = -3.2976
Qtd. tópicos = 10 -> Coerência = -3.5563
Qtd. tópicos = 12 -> Coerência = -5.2403
Qtd. tópicos = 14 -> Coerência = -4.4918
Qtd. tópicos = 16 -> Coerência = -3.9689
Qtd. tóp

In [ ]:
# Escolha da métrica
chosen_metric = 'c_npmi'  # ou 'c_v', 'u_mass', 'c_uci'

# Obter o melhor modelo treinado
best_index = ptm_coherence_results[chosen_metric]['coherence_values'].index(
    max(ptm_coherence_results[chosen_metric]['coherence_values'])
)
best_model = ptm_coherence_results[chosen_metric]['model_list'][best_index]

# Gerar DataFrame com tópicos
def get_topics_dataframe(model, top_n=10):
    topics_data = []
    for topic_id in range(model.k):
        words_probs = model.get_topic_words(topic_id, top_n=top_n)
        topic_words = [word for word, prob in words_probs]
        topic_probs = [prob for word, prob in words_probs]
        topics_data.append({
            'Tópico': topic_id,
            'Palavras': ', '.join(topic_words),
            'Pesos': topic_probs
        })
    return pd.DataFrame(topics_data)

df_topicos = get_topics_dataframe(best_model)

df_topicos.to_csv('./resultados_pseudo/wpp/pseudo_wpp_politico_c_npmi.csv', index=False)

div = topic_diversity(best_model)
print(f"Diversidade de Tópicos: {round(div, 4)}")

irbo_score = compute_irbo_ptm(best_model)
print(f"IRBO médio entre tópicos: {round(irbo_score, 4)}")

df_topicos.head(10)

Diversidade de Tópicos: 0.95
IRBO médio entre tópicos: 0.7071


,Tópico,Palavras,Pesos
0,0,"deus, senhor, jesus, nome, vida, cristo, porqu...","[0.037812601774930954, 0.024820925667881966, 0..."
1,1,"deus, todos, vamos, brasil, fe, acima, amor, f...","[0.06376680731773376, 0.017140470445156097, 0...."


In [ ]:
# Escolha da métrica
chosen_metric = 'c_v'  # ou 'c_v', 'u_mass', 'c_uci'

# Obter o melhor modelo treinado
best_index = ptm_coherence_results[chosen_metric]['coherence_values'].index(
    max(ptm_coherence_results[chosen_metric]['coherence_values'])
)
best_model = ptm_coherence_results[chosen_metric]['model_list'][best_index]

# Gerar DataFrame com tópicos
def get_topics_dataframe(model, top_n=10):
    topics_data = []
    for topic_id in range(model.k):
        words_probs = model.get_topic_words(topic_id, top_n=top_n)
        topic_words = [word for word, prob in words_probs]
        topic_probs = [prob for word, prob in words_probs]
        topics_data.append({
            'Tópico': topic_id,
            'Palavras': ', '.join(topic_words),
            'Pesos': topic_probs
        })
    return pd.DataFrame(topics_data)

df_topicos = get_topics_dataframe(best_model)

df_topicos.to_csv('./resultados_pseudo/wpp/pseudo_wpp_politico_c_v.csv', index=False)

div = topic_diversity(best_model)
print(f"Diversidade de Tópicos: {round(div, 4)}")

irbo_score = compute_irbo_ptm(best_model)
print(f"IRBO médio entre tópicos: {round(irbo_score, 4)}")

df_topicos.head(10)

Diversidade de Tópicos: 0.95
IRBO médio entre tópicos: 0.7071


,Tópico,Palavras,Pesos
0,0,"deus, senhor, jesus, nome, vida, cristo, porqu...","[0.037812601774930954, 0.024820925667881966, 0..."
1,1,"deus, todos, vamos, brasil, fe, acima, amor, f...","[0.06376680731773376, 0.017140470445156097, 0...."


In [ ]:
# Escolha da métrica
chosen_metric = 'u_mass'  # ou 'c_v', 'u_mass', 'c_uci'

# Obter o melhor modelo treinado
best_index = ptm_coherence_results[chosen_metric]['coherence_values'].index(
    max(ptm_coherence_results[chosen_metric]['coherence_values'])
)
best_model = ptm_coherence_results[chosen_metric]['model_list'][best_index]

# Gerar DataFrame com tópicos
def get_topics_dataframe(model, top_n=10):
    topics_data = []
    for topic_id in range(model.k):
        words_probs = model.get_topic_words(topic_id, top_n=top_n)
        topic_words = [word for word, prob in words_probs]
        topic_probs = [prob for word, prob in words_probs]
        topics_data.append({
            'Tópico': topic_id,
            'Palavras': ', '.join(topic_words),
            'Pesos': topic_probs
        })
    return pd.DataFrame(topics_data)

df_topicos = get_topics_dataframe(best_model)

df_topicos.to_csv('./resultados_pseudo/wpp/pseudo_wpp_politico_u_mass.csv', index=False)

div = topic_diversity(best_model)
print(f"Diversidade de Tópicos: {round(div, 4)}")

irbo_score = compute_irbo_ptm(best_model)
print(f"IRBO médio entre tópicos: {round(irbo_score, 4)}")

df_topicos.head(10)

Diversidade de Tópicos: 0.95
IRBO médio entre tópicos: 0.7071


,Tópico,Palavras,Pesos
0,0,"deus, senhor, jesus, nome, vida, cristo, porqu...","[0.037812601774930954, 0.024820925667881966, 0..."
1,1,"deus, todos, vamos, brasil, fe, acima, amor, f...","[0.06376680731773376, 0.017140470445156097, 0...."


In [ ]:
# Escolha da métrica
chosen_metric = 'c_uci'  # ou 'c_v', 'u_mass', 'c_uci'

# Obter o melhor modelo treinado
best_index = ptm_coherence_results[chosen_metric]['coherence_values'].index(
    max(ptm_coherence_results[chosen_metric]['coherence_values'])
)
best_model = ptm_coherence_results[chosen_metric]['model_list'][best_index]

# Gerar DataFrame com tópicos
def get_topics_dataframe(model, top_n=10):
    topics_data = []
    for topic_id in range(model.k):
        words_probs = model.get_topic_words(topic_id, top_n=top_n)
        topic_words = [word for word, prob in words_probs]
        topic_probs = [prob for word, prob in words_probs]
        topics_data.append({
            'Tópico': topic_id,
            'Palavras': ', '.join(topic_words),
            'Pesos': topic_probs
        })
    return pd.DataFrame(topics_data)

df_topicos = get_topics_dataframe(best_model)

df_topicos.to_csv('./resultados_pseudo/wpp/pseudo_wpp_politico_c_uci.csv', index=False)

div = topic_diversity(best_model)
print(f"Diversidade de Tópicos: {round(div, 4)}")

irbo_score = compute_irbo_ptm(best_model)
print(f"IRBO médio entre tópicos: {round(irbo_score, 4)}")

df_topicos.head(10)

Diversidade de Tópicos: 0.95
IRBO médio entre tópicos: 0.7071


,Tópico,Palavras,Pesos
0,0,"deus, senhor, jesus, nome, vida, cristo, porqu...","[0.037812601774930954, 0.024820925667881966, 0..."
1,1,"deus, todos, vamos, brasil, fe, acima, amor, f...","[0.06376680731773376, 0.017140470445156097, 0...."
